<a href="https://colab.research.google.com/github/PaoloGerosa/Chemotherapy-Associated-Liver-Injury/blob/main/Kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Packages

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats
from scipy import ndimage
from skimage import data
from skimage.measure import shannon_entropy
from skimage.filters.rank import entropy as f_entropy
from skimage.morphology import disk, ball
from statistics import mean
from statistics import variance 

import plotly.express as px
import plotly.graph_objects as go


# Calcolo Kernel

In [ ]:
def abs_diff_metric(pixels):
    sum = 0
    center = len(pixels)//2
    for i in range(len(pixels)):
      sum += np.abs(pixels[i]-pixels[center])
    return sum

def make_filters(df, kernel_dim):
  
  stat_features = {}
  stat_quantities = {}
  histograms = {}
  second_histograms = {}

  for name, sample in df.iterrows():
    _value = sample.VOI
    value = np.array(_value)

    # median filter (= median over kernels) and histograms
    median_filter = ndimage.median_filter(value, size=kernel_dim, mode='reflect')
    median_filter_hist = np.histogram(median_filter, bins=20)
    median_filter_second_hist = ndimage.gaussian_laplace(median_filter, sigma=1, mode='reflect')
    median_filter_second_hist = np.histogram(median_filter_second_hist, bins=100)

    # dispersion filter (= variance over kernels) and histograms
    win_mean = ndimage.uniform_filter(value, size=kernel_dim)
    win_sqr_mean = ndimage.uniform_filter(value**2, size=kernel_dim)
    dispersion = win_sqr_mean - win_mean**2
    dispersion_filter_hist = np.histogram(dispersion, bins=20)
    dispersion_filter_second_hist = ndimage.gaussian_laplace(dispersion, sigma=1, mode='reflect')
    dispersion_filter_second_hist = np.histogram(dispersion_filter_second_hist, bins=100)

    # (max - min) filter and histograms
    max_filter = ndimage.maximum_filter(value,size=(kernel_dim,kernel_dim,kernel_dim),mode='reflect')
    min_filter = ndimage.minimum_filter(value,size=(kernel_dim,kernel_dim,kernel_dim),mode='reflect')
    max_min_filter = max_filter - min_filter
    max_min_filter_hist = np.histogram(max_min_filter, bins=20)
    max_min_filter_second_hist = ndimage.gaussian_laplace(max_min_filter, sigma=1, mode='reflect')
    max_min_filter_second_hist = np.histogram(max_min_filter_second_hist, bins=100)

    # abs diff filter and histograms
    abs_diff_filter = ndimage.generic_filter(value,abs_diff_metric,size=(kernel_dim,kernel_dim,kernel_dim),mode='reflect')
    abs_diff_filter_hist = np.histogram(abs_diff_filter, bins=20)
    abs_diff_filter_second_hist = ndimage.gaussian_laplace(abs_diff_filter, sigma=1, mode='reflect')
    abs_diff_filter_second_hist = np.histogram(abs_diff_filter_second_hist, bins=100)

    # mean filter and histograms
    mean_filter = ndimage.generic_filter(value,scipy.ndimage.mean,size=(kernel_dim,kernel_dim,kernel_dim),mode='reflect')
    mean_filter_hist = np.histogram(mean_filter, bins=20)
    mean_filter_second_hist = ndimage.gaussian_laplace(mean_filter, sigma=1, mode='reflect')
    mean_filter_second_hist = np.histogram(mean_filter_second_hist, bins=100)
        
    # autocorrelation filter and histograms
    weights = [[[0,0,0], [0,1,0], [0,0,0]],
              [[0,1,0], [1,1,1], [0,1,0]],
              [[0,0,0], [0,1,0], [0,0,0]]]

    autocorrelation = ndimage.correlate(value, weights, mode='reflect')
    autocorrelation_filter_hist = np.histogram(autocorrelation, bins=20)
    autocorrelation_filter_second_hist = ndimage.gaussian_laplace(autocorrelation, sigma=1, mode='reflect')
    autocorrelation_filter_second_hist = np.histogram(autocorrelation_filter_second_hist, bins=100)
    
    # sobel filter and histograms
    sobel_filter = ndimage.sobel(value, axis=-1, mode='reflect')
    sobel_filter_hist = np.histogram(sobel_filter, bins=20)
    sobel_filter_second_hist = ndimage.gaussian_laplace(sobel_filter, sigma=1, mode='reflect')
    sobel_filter_second_hist = np.histogram(sobel_filter_second_hist, bins=100)
    
    # laplace filter and histograms
    laplace_filter = ndimage.laplace(value, mode='reflect')
    laplace_filter_hist = np.histogram(laplace_filter, bins=20)
    laplace_filter_second_hist = ndimage.gaussian_laplace(laplace_filter, sigma=1, mode='reflect')
    laplace_filter_second_hist = np.histogram(laplace_filter_second_hist, bins=100)

    # prewitt filter and histograms
    prewitt_filter = ndimage.prewitt(value, axis=-1, mode='reflect')
    prewitt_filter_hist = np.histogram(prewitt_filter, bins=20)
    prewitt_filter_second_hist = ndimage.gaussian_laplace(prewitt_filter, sigma=1, mode='reflect')
    prewitt_filter_second_hist = np.histogram(prewitt_filter_second_hist, bins=100)
       
    #global_entropy
    global_entropy = shannon_entropy(value, base=2)

    #local_entropy
    value_converted=np.array(_value, dtype=np.uint16)
    local_entropy = f_entropy(value_converted, ball(5))

    # histogram features
    hist = np.histogram(value, bins=20)
    hist_dist = scipy.stats.rv_histogram(hist)
    m, v, s, k = hist_dist.stats(moments='mvsk')
    entropy = hist_dist.entropy()
    fifth_m = hist_dist.moment(n=5)

    # 2-order histogram features
    second = ndimage.gaussian_laplace(value, sigma=1, mode='reflect')
    second_hist = np.histogram(second, bins=100)
    second_hist_dist = scipy.stats.rv_histogram(second_hist)

    # mean
    avg = [mean(lista) for liste in value for lista in liste]
    avg = mean(avg)

    # variance
    var = value.var()
    

    stat_quantity = { 'Mean' : avg, 
                      'Variance' : var,
                    }
                    
    stat_quantities[name] = stat_quantity

    stat_feature = {'Median filter': median_filter,
                    'Dispersion': dispersion,
                    'Max Min': max_min_filter,
                    'Abs Diff': abs_diff_filter,
                    'Mean': mean_filter,
                    'Autocorrelation': autocorrelation,
                    'Sobel': sobel_filter,
                    'Laplace': laplace_filter,
                    'Prewitt': prewitt_filter,
                    'Global entropy': global_entropy,
                    'Local entropy': local_entropy,
                                        'Second Histogram': second_hist,
                    }

    stat_features[name] = stat_feature

    histogram = {'Volumes': hist,
                 'Histogram statistics': [m, v, s, k, entropy, fifth_m],
                 'Median filter': median_filter_hist,
                 'Dispersion': dispersion_filter_hist,
                 'Max Min': max_min_filter_hist,
                 'Abs Diff': abs_diff_filter_hist,
                 'Mean': mean_filter_hist,
                 'Autocorrelation': autocorrelation_filter_hist,
                 'Sobel': sobel_filter_hist,
                 'Laplace': laplace_filter_hist,
                 'Prewitt': prewitt_filter_hist,
                 }

    histograms[name] = histogram           
                 
    second_histogram = {'Volumes': second_hist,
                        'Median filter': median_filter_second_hist,
                        'Dispersion': dispersion_filter_second_hist,
                        'Max Min': max_min_filter_second_hist,
                        'Abs Diff': abs_diff_filter_second_hist,
                        'Mean': mean_filter_second_hist,
                        'Autocorrelation': autocorrelation_filter_second_hist,
                        'Sobel': sobel_filter_second_hist,
                        'Laplace': laplace_filter_second_hist,
                        'Prewitt': prewitt_filter_second_hist,
                        }  

    second_histograms[name] = second_histogram                        

  stat_quantities = pd.DataFrame.from_dict(stat_quantities, orient = 'index')
  stat_features = pd.DataFrame.from_dict(stat_features, orient='index')
  histograms = pd.DataFrame.from_dict(histograms, orient='index')
  second_histograms = pd.DataFrame.from_dict(second_histograms, orient='index')
  
  # entropy of filters 
  no_filters = {'Global entropy', 'Local entropy','Histogram','Histogram statistics', 'Second Histogram' };
  entropy_of_filters = {};
  for colName, colData in stat_features.iteritems():
    if colName not in no_filters: 
      entropies = [];
      for elem in stat_features[colName]:
        entropies.append(shannon_entropy(np.array(elem),base = 2))
      entropy_of_filters[colName] = entropies

  entropy_of_filters = pd.DataFrame.from_dict(entropy_of_filters, orient = 'index')

  return stat_features, entropy_of_filters, stat_quantities, histograms, second_histograms


'\n\tstat_features: dataframe containing, for each patient\n\t- Median filter\n\t- Dispersion\n  - Max Min\n  - Abs Diff\n  - Mean\n  - Autocorrelation\n  - Sobel\n  - Laplace\n  - Prewitt\n  - Global entropy\n  - Local entropy\n\t- Histogram\n\t- Histogram statistics\n\t- Second Histogram\n'

# **1-D entropy functions** 

In [ ]:
# make_1D_functions(dataset)
# Input: volumes from json file
# Output: - entropy function along x-axis
#         - entropy function along y-axis
#         - entropy function along z-axis
#         - variance function along x-axis
#         - first differences function derived by entropy along x-axis
#         - entropy function along x-axis cutted by one pixel on each side
#         - entropy function along x-axis cutted by three pixels on each side
#         - total variation of the entropy function along x-axis
# Each function is obtained by computing a scalar index for each surface of the volume along a particular direction
# The output is organized in a dataframe, where the rows are the patients with their identificative code and the columns are the functions computed

def make_1D_functions(dataset):
  entropy_functions={}
  

  for name, sample in dataset.iterrows():
      _value = sample.VOI
      value = np.array(_value)
      entropy_x=[]
      entropy_y=[]
      entropy_z=[]
      der_entr_x=[]
      entropy_x_cut=[]
      var = []
      var_tot_x=0
      
      for i in range(30):
          entropy_x.append( shannon_entropy(value[:,:,i], base=2));
          var.append(value[:,:,i].var())

      for j in range(10):
          entropy_y.append( shannon_entropy(value[:,j,:], base=2));
      for z in range(10):
            entropy_z.append( shannon_entropy(value[z,:,:], base=2));
      for i in range(29):
          der_entr_x.append(entropy_x[i+1]-entropy_x[i])
          var_tot_x=var_tot_x+ abs(entropy_x[i+1]-entropy_x[i])
      entropy_x_cut_small=entropy_x[1:29]
      entropy_x_cut_large=entropy_x[3:27]

      stat_feature = {'entropy_x' : entropy_x,
                      'entropy_y' : entropy_y,
                      'entropy_z' : entropy_z,
                      'variance_x': var,
                      'der_entr_x': der_entr_x,
                      'entropy_x_cut_small': entropy_x_cut_small,
                      'entropy_x_cut_large': entropy_x_cut_large,
                      'var_tot_x' : var_tot_x,
                      }
      entropy_functions[name] = stat_feature
      
  entropy_functions = pd.DataFrame.from_dict(entropy_functions, orient='index')
  return entropy_functions


